In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Ensure Jinja2 is installed
try:
    import jinja2
except ImportError:
    !pip install jinja2

# Initial assumptions for the seed round
seed_investment = 8.12 * 1e6  # $8.12 million
seed_ownership = 0.20  # 20% equity ownership

# Founder ownership post-seed round
founder_ownership_post_seed = 1 - seed_ownership

# Scenario options for Series A
scenarios_a = {
    "Scenario 1 ($25M)": 25000000,
    "Scenario 2 ($35M)": 35000000,
    "Scenario 3 ($45M)": 45000000
}

# Widgets for user inputs
scenario_a_widget = widgets.Dropdown(
    options=scenarios_a.keys(),
    value="Scenario 1 ($25M)",
    description='Series A Scenario:',
    style={'description_width': 'initial'}
)

investment_a_widget = widgets.FloatSlider(
    value=10.00,
    min=1.00,
    max=20.00,
    step=0.10,
    description='Series A Investment $ (M):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

dilution_a_widget = widgets.FloatSlider(
    value=0.50,
    min=0,
    max=1,
    step=0.05,
    description='Series A Dilution:',
    continuous_update=False,
    style={'description_width': 'initial'}
)

# Additional rounds (Series B, C, and D)
investment_b_widget = widgets.FloatSlider(
    value=15.00,
    min=1.00,
    max=25.00,
    step=0.10,
    description='Series B Investment $ (M):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

follow_on_b_widget = widgets.FloatSlider(
    value=10.00,  # Default set to initial Series A investment amount
    min=0.00,
    max=20.00,
    step=0.10,
    description='Follow-On Series B $ (M):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

pre_money_b_widget = widgets.FloatSlider(
    value=50.00,
    min=10.00,
    max=100.00,
    step=1.00,
    description='Series B Pre-Money $ (M):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

dilution_b_widget = widgets.FloatSlider(
    value=0.50,
    min=0,
    max=1,
    step=0.05,
    description='Series B Dilution:',
    continuous_update=False,
    style={'description_width': 'initial'}
)

investment_c_widget = widgets.FloatSlider(
    value=20.00,
    min=1.00,
    max=30.00,
    step=0.10,
    description='Series C Investment $ (M):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

follow_on_c_widget = widgets.FloatSlider(
    value=10.00,  # Default set to initial Series A investment amount
    min=0.00,
    max=20.00,
    step=0.10,
    description='Follow-On Series C $ (M):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

pre_money_c_widget = widgets.FloatSlider(
    value=100.00,
    min=20.00,
    max=200.00,
    step=1.00,
    description='Series C Pre-Money $ (M):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

dilution_c_widget = widgets.FloatSlider(
    value=0.50,
    min=0,
    max=1,
    step=0.05,
    description='Series C Dilution:',
    continuous_update=False,
    style={'description_width': 'initial'}
)

investment_d_widget = widgets.FloatSlider(
    value=25.00,
    min=1.00,
    max=50.00,
    step=0.10,
    description='Series D Investment $ (M):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

pre_money_d_widget = widgets.FloatSlider(
    value=150.00,
    min=50.00,
    max=300.00,
    step=1.00,
    description='Series D Pre-Money $ (M):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

dilution_d_widget = widgets.FloatSlider(
    value=0.50,
    min=0,
    max=1,
    step=0.05,
    description='Series D Dilution:',
    continuous_update=False,
    style={'description_width': 'initial'}
)

return_multiple_widget = widgets.SelectionSlider(
    options=[1, 2, 3, 5],
    value=1,
    description='Return x:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    style={'description_width': 'initial'}
)

# Function to calculate post-dilution ownership
def calculate_post_dilution_ownership(initial_ownership, dilution):
    return initial_ownership * (1 - dilution)

# Function to calculate necessary exit valuation
def calculate_exit_valuation(investment_amount, ownership_percentage, target_return):
    return (target_return * investment_amount) / ownership_percentage

# Function to update the results based on user inputs
def update_model(scenario_a, investment_amount_a_m, dilution_a, investment_amount_b_m, follow_on_b_m, pre_money_b_m, dilution_b, investment_amount_c_m, follow_on_c_m, pre_money_c_m, dilution_c, investment_amount_d_m, pre_money_d_m, dilution_d, return_multiple):
    investment_amount_a = investment_amount_a_m * 1e6  # Convert millions to actual value
    follow_on_b = follow_on_b_m * 1e6  # Convert millions to actual value
    follow_on_c = follow_on_c_m * 1e6  # Convert millions to actual value
    follow_on_d = investment_amount_a  # Follow-on set to initial investment amount
    pre_money_valuation_a = scenarios_a[scenario_a]
    post_money_valuation_a = pre_money_valuation_a + investment_amount_a
    series_a_ownership = investment_amount_a / post_money_valuation_a
    
    # Calculate ownership post Series A considering initial dilution from seed round
    founder_ownership_post_series_a = founder_ownership_post_seed * (pre_money_valuation_a / post_money_valuation_a)
    seed_ownership_post_series_a = seed_ownership * (pre_money_valuation_a / post_money_valuation_a)
    post_dilution_ownership_a = calculate_post_dilution_ownership(series_a_ownership, dilution_a)
    
    # Series B calculations
    investment_amount_b = investment_amount_b_m * 1e6
    pre_money_valuation_b = pre_money_b_m * 1e6
    post_money_valuation_b = pre_money_valuation_b + investment_amount_b + follow_on_b
    series_b_ownership = (investment_amount_b + follow_on_b) / post_money_valuation_b
    founder_ownership_post_series_b = founder_ownership_post_series_a * (pre_money_valuation_b / post_money_valuation_b)
    seed_ownership_post_series_b = seed_ownership_post_series_a * (pre_money_valuation_b / post_money_valuation_b)
    post_dilution_ownership_b = calculate_post_dilution_ownership(post_dilution_ownership_a, dilution_b)
    
    # Series C calculations
    investment_amount_c = investment_amount_c_m * 1e6
    pre_money_valuation_c = pre_money_c_m * 1e6
    post_money_valuation_c = pre_money_valuation_c + investment_amount_c + follow_on_c
    series_c_ownership = (investment_amount_c + follow_on_c) / post_money_valuation_c
    founder_ownership_post_series_c = founder_ownership_post_series_b * (pre_money_valuation_c / post_money_valuation_c)
    seed_ownership_post_series_c = seed_ownership_post_series_b * (pre_money_valuation_c / post_money_valuation_c)
    post_dilution_ownership_c = calculate_post_dilution_ownership(post_dilution_ownership_b, dilution_c)

    # Series D calculations
    investment_amount_d = investment_amount_d_m * 1e6
    pre_money_valuation_d = pre_money_d_m * 1e6
    post_money_valuation_d = pre_money_valuation_d + investment_amount_d + follow_on_d
    series_d_ownership = (investment_amount_d + follow_on_d) / post_money_valuation_d
    founder_ownership_post_series_d = founder_ownership_post_series_c * (pre_money_valuation_d / post_money_valuation_d)
    seed_ownership_post_series_d = seed_ownership_post_series_c * (pre_money_valuation_d / post_money_valuation_d)
    post_dilution_ownership_d = calculate_post_dilution_ownership(post_dilution_ownership_c, dilution_d)

    # Necessary exit valuation at IPO
    total_investment = investment_amount_a + investment_amount_b + follow_on_b + investment_amount_c + follow_on_c + investment_amount_d + follow_on_d
    necessary_exit_valuation = calculate_exit_valuation(total_investment, post_dilution_ownership_d, return_multiple)

    # Create DataFrame for the result
    result_df = pd.DataFrame([
        {
            'Round': 'Seed',
            'Investment': f"${seed_investment/1e6:,.2f}M",
            'Post-Money Valuation': 'N/A',
            'Dilution Level': 'N/A',
            'Ownership %': seed_ownership * 100
        },
        {
            'Round': 'Series A',
            'Investment': f"${investment_amount_a/1e6:,.2f}M",
            'Post-Money Valuation': f"${post_money_valuation_a/1e6:,.2f}M",
            'Dilution Level': f"{int(dilution_a * 100)}%",
            'Ownership %': series_a_ownership * 100
        },
        {
            'Round': 'Series B',
            'Investment': f"${investment_amount_b/1e6:,.2f}M",
            'Follow-On': f"${follow_on_b/1e6:,.2f}M",
            'Post-Money Valuation': f"${post_money_valuation_b/1e6:,.2f}M",
            'Dilution Level': f"{int(dilution_b * 100)}%",
            'Ownership %': series_b_ownership * 100
        },
        {
            'Round': 'Series C',
            'Investment': f"${investment_amount_c/1e6:,.2f}M",
            'Follow-On': f"${follow_on_c/1e6:,.2f}M",
            'Post-Money Valuation': f"${post_money_valuation_c/1e6:,.2f}M",
            'Dilution Level': f"{int(dilution_c * 100)}%",
            'Ownership %': series_c_ownership * 100
        },
        {
            'Round': 'Series D',
            'Investment': f"${investment_amount_d/1e6:,.2f}M",
            'Follow-On': f"${follow_on_d/1e6:,.2f}M",
            'Post-Money Valuation': f"${post_money_valuation_d/1e6:,.2f}M",
            'Dilution Level': f"{int(dilution_d * 100)}%",
            'Ownership %': series_d_ownership * 100
        },
        {
            'Round': 'Post-IPO',
            'Investment': 'N/A',
            'Post-Money Valuation': f"${necessary_exit_valuation/1e6:,.2f}M",
            'Dilution Level': 'N/A',
            'Ownership %': post_dilution_ownership_d * 100
        }
    ])

    # Highlight each series with different colors
    def highlight_rounds(row):
        if row['Round'] == 'Seed':
            return ['background-color: lightblue'] * len(row)
        elif row['Round'] == 'Series A':
            return ['background-color: lightgreen'] * len(row)
        elif row['Round'] == 'Series B':
            return ['background-color: lightyellow'] * len(row)
        elif row['Round'] == 'Series C':
            return ['background-color: lightcoral'] * len(row)
        elif row['Round'] == 'Series D':
            return ['background-color: lightpink'] * len(row)
        else:
            return ['background-color: lightgrey'] * len(row)

    # Display the result
    styled_df = result_df.style.apply(highlight_rounds, axis=1)
    display(styled_df)

# Interactive output
output = widgets.interactive_output(update_model, {
    'scenario_a': scenario_a_widget,
    'investment_amount_a_m': investment_a_widget,
    'dilution_a': dilution_a_widget,
    'investment_amount_b_m': investment_b_widget,
    'follow_on_b_m': follow_on_b_widget,
    'pre_money_b_m': pre_money_b_widget,
    'dilution_b': dilution_b_widget,
    'investment_amount_c_m': investment_c_widget,
    'follow_on_c_m': follow_on_c_widget,
    'pre_money_c_m': pre_money_c_widget,
    'dilution_c': dilution_c_widget,
    'investment_amount_d_m': investment_d_widget,
    'pre_money_d_m': pre_money_d_widget,
    'dilution_d': dilution_d_widget,
    'return_multiple': return_multiple_widget
})

# Create a box layout for widgets
widgets_box = widgets.VBox([
    scenario_a_widget, investment_a_widget, dilution_a_widget,
    investment_b_widget, follow_on_b_widget, pre_money_b_widget, dilution_b_widget,
    investment_c_widget, follow_on_c_widget, pre_money_c_widget, dilution_c_widget,
    investment_d_widget, pre_money_d_widget, dilution_d_widget,
    return_multiple_widget
])

# Display widgets and output
display(widgets_box, output)


Output()